In [41]:
import json
from tqdm import tqdm
from transformers import AutoTokenizer  

In [42]:
MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct" 
INPUT_PATH = "final_results.json"
OUTPUT_PATH = "final_results_with_token_index.json"
LOG_EVERY_N = int(1e3) 

In [43]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
print(f"Loaded tokenizer: {MODEL_NAME}  (vocab = {len(tokenizer)})")

Loaded tokenizer: meta-llama/Llama-3.2-3B-Instruct  (vocab = 128256)


In [44]:
with open(INPUT_PATH, "r", encoding="utf8") as f:
    data = json.load(f)
print(f"Loaded {len(data)} records from {INPUT_PATH}")

Loaded 18317 records from final_results.json


In [45]:
def find_token_start(big_ids, sub_ids):
    """Return first index where sub_ids appears in big_ids, else -1."""
    if not sub_ids: 
        return -1
    sub_len = len(sub_ids)
    for i in range(len(big_ids) - sub_len + 1):
        if big_ids[i : i + sub_len] == sub_ids:
            return i
    return -1

results = []
for idx, elem in enumerate(tqdm(data, desc="Processing")):
    if not elem["is_correct"]:
        print(f"Skipping idx={idx} (not correct)")
        continue

    src = elem["source_text"].replace("$", "")
    query = elem["extracted_search_term"].replace("$", "")

    ids_full = tokenizer.encode(src, add_special_tokens=False)
    ids_sub = tokenizer.encode(query, add_special_tokens=False)

    start_idx = find_token_start(ids_full, ids_sub)
    if start_idx == -1:
        print(f"No token match at idx={idx}, term='{query}'")
        continue         

    out_elem = dict(elem)        

    out_elem["source_text"] = src
    out_elem["extracted_search_term"] = query

    out_elem["start_index"] = start_idx
    out_elem["end_index"] = start_idx + len(ids_sub) - 1
    out_elem["decoded_value"] = tokenizer.decode(ids_full[start_idx:out_elem["end_index"] + 1], skip_special_tokens=True)

    out_elem["tokens"] = ids_full
    results.append(out_elem)

    if (len(results) % LOG_EVERY_N) == 0:
        print(f"Processed OK: {len(results)} items retained")
    
    '''if idx == 32:
        break'''

print(f"Finished. Retained {len(results)} / {len(data)} items in total.")

Processing:   0%|          | 0/18317 [00:00<?, ?it/s]

Skipping idx=49 (not correct)
Skipping idx=50 (not correct)
No token match at idx=74, term='.4 liters * 1000 ml/liter'
Skipping idx=86 (not correct)


Processing:   1%|          | 138/18317 [00:00<00:13, 1376.24it/s]

Skipping idx=187 (not correct)
Skipping idx=189 (not correct)


Processing:   2%|▏         | 276/18317 [00:00<00:13, 1374.36it/s]

...

No token match at idx=17402, term='8(100)'
Skipping idx=17409 (not correct)
Skipping idx=17447 (not correct)
Skipping idx=17557 (not correct)
Skipping idx=17597 (not correct)
Skipping idx=17603 (not correct)
No token match at idx=17616, term='.75 * 4'
Skipping idx=17689 (not correct)
No token match at idx=17712, term='.20*350'


Processing:  99%|█████████▉| 18092/18317 [00:12<00:00, 1469.64it/s]

Skipping idx=17838 (not correct)
No token match at idx=17851, term='(4+2)+3'
Skipping idx=17872 (not correct)
Skipping idx=18064 (not correct)


Processing: 100%|██████████| 18317/18317 [00:12<00:00, 1428.63it/s]

Skipping idx=18240 (not correct)
Skipping idx=18250 (not correct)
Skipping idx=18251 (not correct)
Skipping idx=18289 (not correct)
Processed OK: 18000 items retained
Skipping idx=18315 (not correct)
Finished. Retained 18005 / 18317 items in total.


In [46]:
with open(OUTPUT_PATH, "w", encoding="utf8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"Output written ➜  {OUTPUT_PATH}")

Output written ➜  final_results_with_token_index.json
